# Generate PMTiles 

Take all the Parquet files, generate their FlatGeoBuff versions and generate regional PMTiles from those. Then merge all PMTiles to a single file for the final application.

In [2]:
import os
import tempfile

import geopandas as gpd
from tqdm.auto import tqdm

In [3]:
pmtiles_destination = "/data/uscuni-ulce/pmtiles/"

In [4]:
# os.mkdir(pmtiles_destination)
# os.mkdir(pmtiles_destination + "regions")

In [5]:
regions = gpd.read_parquet("/data/uscuni-ulce/regions/cadastre_regions_hull.parquet")

In [6]:
v = "v3"

In [10]:
blg = gpd.read_parquet(f"{folder}clusters_{69333}_{v}.pq")

In [11]:
blg

,currentUse,buildingNature,heightAboveGround,iid,geometry,label,morph,final,regional,final_without_noise
0,None,None,NaN,80677517,"POLYGON ((4616718.204 3026081.581, 4616720.974...",0,69333_0_-1,-1,-1,84
1,None,None,NaN,80677518,"POLYGON ((4617480.926 3025927.323, 4617486.098...",1775,69333_1_0,84,69333_148,84
2,None,None,NaN,80677519,"POLYGON ((4617331.885 3025855.863, 4617331.941...",1775,69333_1_0,84,69333_148,84
3,None,None,NaN,80677520,"POLYGON ((4617495.899 3025921.454, 4617499.35 ...",1775,69333_1_0,84,69333_148,84
4,None,None,NaN,80677521,"POLYGON ((4617561.862 3026075.057, 4617564.522...",1775,69333_1_0,84,69333_148,84
...,...,...,...,...,...,...,...,...,...,...
446822,None,None,NaN,82307932,"POLYGON ((4643444.536 3006455.727, 4643447.422...",5749,69333_849_640,3,69333_63,3
446823,None,None,NaN,82308166,"POLYGON ((4640285.807 3000016.398, 4640291.15 ...",5146,69333_849_-1,-1,-1,3
446824,None,None,NaN,82308213,"POLYGON ((4644217.544 3009263.369, 4644217.737...",5578,69333_849_487,83,69333_178,83
446825,None,None,NaN,82308225,"POLYGON ((4637968.015 3010358.876, 4637978.019...",5602,69333_849_508,81,69333_110,81


## Buildings

In [7]:
folder = "/data/uscuni-ulce/processed_data/clusters/"

In [9]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}clusters_{region}_{v}.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}regions/{region}.pmtiles -l buildings -pf -pk -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [12]:
os.system(f"tile-join -o {pmtiles_destination}complete.pmtiles -f -pk {pmtiles_destination}regions/*.pmtiles")

0

## Morphotopes

In [13]:
folder = "/data/uscuni-ulce/processed_data/morphotope_clusters/"

In [14]:
# os.mkdir(pmtiles_destination + "morphotopes")

In [15]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}morphotopes/{region}.pmtiles -z 12 -l morphotopes --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [16]:
os.system(f"tile-join -o {pmtiles_destination}morphotopes.pmtiles -f -pk {pmtiles_destination}morphotopes/*.pmtiles")

0

## 250m grid

In [17]:
folder = "/data/uscuni-ulce/grid_250/"

In [18]:
# os.mkdir(pmtiles_destination + "grid_250")

FileExistsError: [Errno 17] File exists: '/data/uscuni-ulce/pmtiles/grid_250'

In [ ]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}grid_250/{region}.pmtiles -z 11 -l grid --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [ ]:
os.system(f"tile-join -o {pmtiles_destination}grid_250.pmtiles -f -pk {pmtiles_destination}grid_250/*.pmtiles")

## H3

In [21]:
folder = "/data/uscuni-ulce/h3/"

In [18]:
# os.mkdir(pmtiles_destination + "h3")

In [22]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}h3/{region}.pmtiles -z 12 -l h3 --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [23]:
os.system(f"tile-join -o {pmtiles_destination}h3.pmtiles -f -pk {pmtiles_destination}h3/*.pmtiles")

0

In [24]:
os.system("aws s3 --profile uscuni cp /data/uscuni-ulce/pmtiles/h3.pmtiles s3://uscuni-public")

upload: ../../../../../data/uscuni-ulce/pmtiles/h3.pmtiles to s3://uscuni-public/h3.pmtiles


0